In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
sys.path.append('../Shared')
import processing, Model, Visualization, modules

def get_label_from_filename(fname):
    for cls, label in class_to_label.items():
        if cls in fname:
            return label
    return None  # if no class found


Label 파일에서 Status랑 Group있는데 걸음걸이 마다 랑 뭐 다르게 준것같음 - 그냥 무시하고 전체 레이블로 가기

In [2]:
classes = ['STC', 'WAK', 'STDUP', 'SITDN', 'UPS', 'DNS', 'KLFT', 'TPTO', 'LLF', 'LLB', 'LLS', 'KLCL', 'HS', 'TO', 'LUGF', 'LUGB'] #16 classes

Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

base_path = 'D:/Data/Gait-EMG/SIAT/SIAT_LLMD20230404/'
Sub_lst_data = [f"Sub{i:02d}"+"/Data/" for i in range(1, 41)]
Sub_lst_label = [f"Sub{i:02d}"+"/Labels/" for i in range(1, 41)]

In [3]:
###나중에 여기부터 for (Subject 단위) - 지금부터 1 subject 시작
data_lst = os.listdir(base_path + Sub_lst_data[0])
label_lst = os.listdir(base_path + Sub_lst_label[0])
#label = pd.read_csv(base_path + Sub_lst_label[0] + label_lst[0])

class_to_label = {cls: i for i, cls in enumerate(classes)}

data_lst = os.listdir(base_path + Sub_lst_data[0])
label_lst = os.listdir(base_path + Sub_lst_label[0])

dataset, labels = [], []
for l, d in zip(label_lst, data_lst):
    print(l, d)
    data = pd.read_csv(base_path + Sub_lst_data[0] + "/" + d)
    data = data.iloc[:, 1:]   # 첫 column(time) 제거

    label = get_label_from_filename(l)

    labels.append(np.array([label] * data.shape[0]))
    dataset.append(data)
    print(np.array([label] * data.shape[0]).shape, data.shape)

#pd.Series(labels[0]).value_counts()
D = np.concatenate(dataset, axis=0)
L = np.concatenate(labels, axis=0)
print(D.shape, L.shape)

balanced_idx = modules.Downsample_to_balance_class(L)
X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
print("EMG:", X_emg_bal.shape, "y:", y_bal.shape)

window_size, step_size = 400, 20  #1926Hz
num_channels, num_features = X_emg_bal.shape[1], 30

X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal, window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
all_y = modules.y_change_to_int(all_y)
all_X = all_X.reshape(-1, num_channels, num_features, 1)


from sklearn.model_selection import train_test_split

Sub_acc = []
for feature_idx in range(num_features):
    X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
    X_feature = X_feature.reshape(-1, num_channels, 1)
    X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

    model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

    print(f"\t Feature {feature_idx} ({Features[feature_idx]}): "
        f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
    Sub_acc.append(np.max(history.history['val_accuracy'])*100)

Sub01_DNS_Label.csv Sub01_DNS_Data.csv
(30668,) (30668, 25) 

Sub01_HS_Label.csv Sub01_HS_Data.csv
(94817,) (94817, 25) 

Sub01_KLCL_Label.csv Sub01_KLCL_Data.csv
(101249,) (101249, 25) 

Sub01_KLFT_Label.csv Sub01_KLFT_Data.csv
(104193,) (104193, 25) 

Sub01_LLB_Label.csv Sub01_LLB_Data.csv
(91745,) (91745, 25) 

Sub01_LLF_Label.csv Sub01_LLF_Data.csv
(92417,) (92417, 25) 

Sub01_LLS_Label.csv Sub01_LLS_Data.csv
(103041,) (103041, 25) 

Sub01_LUGB_Label.csv Sub01_LUGB_Data.csv
(101281,) (101281, 25) 

Sub01_LUGF_Label.csv Sub01_LUGF_Data.csv
(98913,) (98913, 25) 

Sub01_SITDN_Label.csv Sub01_SITDN_Data.csv
(84645,) (84645, 25) 

Sub01_STC_Label.csv Sub01_STC_Data.csv
(47845,) (47845, 25) 

Sub01_STDUP_Label.csv Sub01_STDUP_Data.csv
(81925,) (81925, 25) 

Sub01_TO_Label.csv Sub01_TO_Data.csv
(93633,) (93633, 25) 

Sub01_TPTO_Label.csv Sub01_TPTO_Data.csv
(101313,) (101313, 25) 

Sub01_UPS_Label.csv Sub01_UPS_Data.csv
(35479,) (35479, 25) 

Sub01_WAK_Label.csv Sub01_WAK_Data.csv
(27164,

In [5]:
#X_emg_bal, y_bal = modules.random_downsample(X_emg_bal, y_bal, fraction=0.5)
#print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

EMG: (434624, 25) y: (434624,)


In [6]:
window_size, step_size = 400, 20  #1926Hz
num_channels, num_features = X_emg_bal.shape[1], 30

X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal, window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
all_y = modules.y_change_to_int(all_y)
all_X = all_X.reshape(-1, num_channels, num_features, 1)

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\github\Lower-limb\Gait3-SAIT\../Shared\processing.py:85: RuntimeWarning: divide by zero encountered in log
  spectrum = np.log(np.abs(fft(signal)))


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [7]:
from sklearn.model_selection import train_test_split

Sub_acc = []
for feature_idx in range(num_features):
    X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
    X_feature = X_feature.reshape(-1, num_channels, 1)
    X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

    model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

    print(f"\t Feature {feature_idx} ({Features[feature_idx]}): "
        f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
    Sub_acc.append(np.max(history.history['val_accuracy'])*100)

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


	 Feature 0 (mav): val acc => 99.75%
	 Feature 1 (var): val acc => 99.75%
	 Feature 2 (zc): val acc => 99.95%
	 Feature 3 (iemg): val acc => 99.61%
	 Feature 4 (wl): val acc => 99.56%
	 Feature 5 (wamp): val acc => 95.67%
	 Feature 6 (mavs): val acc => 38.08%
	 Feature 7 (rms): val acc => 99.86%
	 Feature 8 (ssc): val acc => 10.78%
	 Feature 9 (msq): val acc => 99.84%
	 Feature 10 (v3): val acc => 99.59%
	 Feature 11 (ld): val acc => 99.93%
	 Feature 12 (dabs): val acc => 99.86%
	 Feature 13 (mfl): val acc => 99.68%
	 Feature 14 (mpr): val acc => 92.06%
	 Feature 15 (mnf): val acc => 60.56%
	 Feature 16 (psr): val acc => 6.19%
	 Feature 17 (arc1): val acc => 66.13%
	 Feature 18 (arc2): val acc => 65.46%
	 Feature 19 (arc3): val acc => 74.67%
	 Feature 20 (arc4): val acc => 56.55%
	 Feature 21 (cc1): val acc => 5.89%
	 Feature 22 (cc2): val acc => 5.53%
	 Feature 23 (cc3): val acc => 5.53%
	 Feature 24 (cca): val acc => 5.53%
	 Feature 25 (dwtc1): val acc => 99.77%
	 Feature 26 (dwtc2):

In [4]:
###나중에 여기부터 for (Subject 단위) - 지금부터 1 subject 시작
ACC_lst = []
for SUB, LAB in zip(Sub_lst_data[34:], Sub_lst_label[34:]):
    print(f"Sub: {SUB} Start\n==================================\n==================================\n")
    data_lst = os.listdir(base_path + SUB)
    label_lst = os.listdir(base_path + LAB)

    class_to_label = {cls: i for i, cls in enumerate(classes)}

    dataset, labels = [], []
    for l, d in zip(label_lst, data_lst):

        data = pd.read_csv(base_path + SUB + "/" + d)
        data = data.iloc[:, 1:]   # 첫 column(time) 제거

        label = get_label_from_filename(l)

        labels.append(np.array([label] * data.shape[0]))
        dataset.append(data)
        print("\t", l, d, " ====> ", np.array([label] * data.shape[0]).shape, data.shape)

    D = np.concatenate(dataset, axis=0)
    L = np.concatenate(labels, axis=0)
    print(D.shape, L.shape)

    balanced_idx = modules.Downsample_to_balance_class(L)
    X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
    print("\t EMG:", X_emg_bal.shape, "y:", y_bal.shape)

    window_size, step_size = 400, 20
    num_channels, num_features = X_emg_bal.shape[1], 30

    X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

    all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
    all_y = modules.y_change_to_int(all_y)
    all_X = all_X.reshape(-1, num_channels, num_features, 1)


    from sklearn.model_selection import train_test_split

    Sub_acc = []
    for feature_idx in range(num_features):
        X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
        X_feature = X_feature.reshape(-1, num_channels, 1)
        X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

        model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

        print(f"\t Feature {feature_idx} ({Features[feature_idx]}): "
            f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
        Sub_acc.append(np.max(history.history['val_accuracy'])*100)
    ACC_lst.append(Sub_acc)

Sub: Sub35/Data/ Start

	 Sub35_DNS_Label.csv Sub35_DNS_Data.csv  ====>  (48359,) (48359, 25)
	 Sub35_HS_Label.csv Sub35_HS_Data.csv  ====>  (105025,) (105025, 25)
	 Sub35_KLCL_Label.csv Sub35_KLCL_Data.csv  ====>  (105121,) (105121, 25)
	 Sub35_KLFT_Label.csv Sub35_KLFT_Data.csv  ====>  (92225,) (92225, 25)
	 Sub35_LLB_Label.csv Sub35_LLB_Data.csv  ====>  (100769,) (100769, 25)
	 Sub35_LLF_Label.csv Sub35_LLF_Data.csv  ====>  (107009,) (107009, 25)
	 Sub35_LLS_Label.csv Sub35_LLS_Data.csv  ====>  (105537,) (105537, 25)
	 Sub35_LUGB_Label.csv Sub35_LUGB_Data.csv  ====>  (97121,) (97121, 25)
	 Sub35_LUGF_Label.csv Sub35_LUGF_Data.csv  ====>  (103041,) (103041, 25)
	 Sub35_SITDN_Label.csv Sub35_SITDN_Data.csv  ====>  (111365,) (111365, 25)
	 Sub35_STC_Label.csv Sub35_STC_Data.csv  ====>  (46117,) (46117, 25)
	 Sub35_STDUP_Label.csv Sub35_STDUP_Data.csv  ====>  (102277,) (102277, 25)
	 Sub35_TO_Label.csv Sub35_TO_Data.csv  ====>  (96449,) (96449, 25)
	 Sub35_TPTO_Label.csv Sub35_TPTO_Data

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\github\Lower-limb\Gait3-SAIT\../Shared\processing.py:85: RuntimeWarning: divide by zero encountered in log
  spectrum = np.log(np.abs(fft(signal)))


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


C:\Users\hml76\PycharmProjects\Cross-motor-decoding\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


	 Feature 0 (mav): val acc => 99.64%
	 Feature 1 (var): val acc => 99.79%
	 Feature 2 (zc): val acc => 99.93%
	 Feature 3 (iemg): val acc => 99.57%
	 Feature 4 (wl): val acc => 99.55%
	 Feature 5 (wamp): val acc => 99.31%
	 Feature 6 (mavs): val acc => 39.16%
	 Feature 7 (rms): val acc => 99.62%
	 Feature 8 (ssc): val acc => 9.67%
	 Feature 9 (msq): val acc => 99.74%
	 Feature 10 (v3): val acc => 99.59%
	 Feature 11 (ld): val acc => 99.67%
	 Feature 12 (dabs): val acc => 99.71%
	 Feature 13 (mfl): val acc => 99.62%
	 Feature 14 (mpr): val acc => 90.12%
	 Feature 15 (mnf): val acc => 78.26%
	 Feature 16 (psr): val acc => 6.04%
	 Feature 17 (arc1): val acc => 65.73%
	 Feature 18 (arc2): val acc => 61.26%
	 Feature 19 (arc3): val acc => 65.25%
	 Feature 20 (arc4): val acc => 51.98%
	 Feature 21 (cc1): val acc => 6.09%
	 Feature 22 (cc2): val acc => 5.44%
	 Feature 23 (cc3): val acc => 5.44%
	 Feature 24 (cca): val acc => 5.44%
	 Feature 25 (dwtc1): val acc => 99.45%
	 Feature 26 (dwtc2): 

...

	 Feature 0 (mav): val acc => 99.12%
	 Feature 1 (var): val acc => 99.63%
	 Feature 2 (zc): val acc => 99.94%
	 Feature 3 (iemg): val acc => 99.63%
	 Feature 4 (wl): val acc => 99.66%
	 Feature 5 (wamp): val acc => 96.71%
	 Feature 6 (mavs): val acc => 37.14%
	 Feature 7 (rms): val acc => 99.33%
	 Feature 8 (ssc): val acc => 8.90%
	 Feature 9 (msq): val acc => 99.54%
	 Feature 10 (v3): val acc => 99.09%
	 Feature 11 (ld): val acc => 99.82%
	 Feature 12 (dabs): val acc => 99.42%
	 Feature 13 (mfl): val acc => 99.60%
	 Feature 14 (mpr): val acc => 88.33%
	 Feature 15 (mnf): val acc => 91.04%
	 Feature 16 (psr): val acc => 6.15%
	 Feature 17 (arc1): val acc => 62.52%
	 Feature 18 (arc2): val acc => 63.50%
	 Feature 19 (arc3): val acc => 61.37%
	 Feature 20 (arc4): val acc => 50.21%
	 Feature 21 (cc1): val acc => 5.70%
	 Feature 22 (cc2): val acc => 6.22%
	 Feature 23 (cc3): val acc => 6.22%
	 Feature 24 (cca): val acc => 6.22%
	 Feature 25 (dwtc1): val acc => 99.36%
	 Feature 26 (dwtc2): 

C:\Users\hml76\PycharmProjects\Cross-motor-decoding\github\Lower-limb\Gait3-SAIT\../Shared\processing.py:85: RuntimeWarning: divide by zero encountered in log
  spectrum = np.log(np.abs(fft(signal)))


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


C:\Users\hml76\PycharmProjects\Cross-motor-decoding\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


	 Feature 0 (mav): val acc => 99.85%
	 Feature 1 (var): val acc => 99.79%
	 Feature 2 (zc): val acc => 99.85%
	 Feature 3 (iemg): val acc => 99.73%
	 Feature 4 (wl): val acc => 99.55%
	 Feature 5 (wamp): val acc => 98.64%
	 Feature 6 (mavs): val acc => 33.03%
	 Feature 7 (rms): val acc => 99.73%
	 Feature 8 (ssc): val acc => 8.47%
	 Feature 9 (msq): val acc => 99.79%
	 Feature 10 (v3): val acc => 99.28%
	 Feature 11 (ld): val acc => 99.79%
	 Feature 12 (dabs): val acc => 99.67%
	 Feature 13 (mfl): val acc => 99.61%
	 Feature 14 (mpr): val acc => 90.48%
	 Feature 15 (mnf): val acc => 76.97%
	 Feature 16 (psr): val acc => 6.12%
	 Feature 17 (arc1): val acc => 47.86%
	 Feature 18 (arc2): val acc => 49.67%
	 Feature 19 (arc3): val acc => 59.58%
	 Feature 20 (arc4): val acc => 63.14%
	 Feature 21 (cc1): val acc => 5.85%
	 Feature 22 (cc2): val acc => 6.30%
	 Feature 23 (cc3): val acc => 6.30%
	 Feature 24 (cca): val acc => 6.30%
	 Feature 25 (dwtc1): val acc => 99.49%
	 Feature 26 (dwtc2): 